In [3]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

%reload_ext jupyternotify

#%notify

<IPython.core.display.Javascript object>

In [199]:
import numpy as np
import sys
from qutip import *
import matplotlib.pyplot as plt
from pathlib import Path

from scipy import linalg as la

parent_path = ".." #str(Path(__file__).resolve().parents[1])
sys.path.append(parent_path)

import pysqkit


import time
import os

from pysqkit.solvers.solvkit import integrate

%matplotlib notebook
plt.rcParams['figure.figsize'] = (10, 7)

In [5]:
from math import pi

h = 6.62607 * 10**(-34)
h_bar = h/(2*pi)
e = 1.6071766*10**(-19)
phi_0 = h/(2*e)


In [81]:
N = 100

# Init

In [82]:
J_c = 300*10**6 *h
lam = .5

In [83]:
t = np.linspace(0, 1000, 100000) * 10**(-9) #in nano seconds

$$\hat{H} = \hat{H}^{(0)}_A + \hat{H}^{(0)}_B + \hat{V} + \hat{H}_{drive}$$

## flxA



$$\hat{H}^{(0)}_A = 4 E_{C,A} \cdot \hat{n}^2_A + \frac{1}{2}E_{L,A}\cdot\hat{\varphi}^2_A - E_{J,A}\cos\left(\hat{\varphi}_A - \phi_{ext,A}\right)$$

In [174]:
e_c_A = 1*10**9 *h
e_l_A = 1.5*10**9 *h
e_j_A = 3.8*10**9 *h
  
flux_A = 1/2  # 1/2 -> pi dans le cos

flxA = pysqkit.qubits.Fluxonium(label = "QBA",
                               charge_energy=e_c_A,
                               induct_energy=e_l_A,
                               joseph_energy=e_j_A,
                               flux=flux_A,
                               basis=None,
                               dim_hilbert=N)

In [175]:
eig_enegies, eig_states = flxA.eig_states(levels=5)
flxA.basis.transform(eig_states)

In [176]:
#Bare energies

bare_E_A = flxA.eig_energies()
bare_E_A/(h*10**9)

array([ 3.12560598,  4.2772998 ,  7.55777961, 10.81091566, 14.50310188,
       18.35763955, 22.30068932, 26.25193133, 30.14840206, 33.93656752])

In [177]:
# transitions freqs

omega_01_A = bare_E_A[1] - bare_E_A[0]
omega_12_A = bare_E_A[2] - bare_E_A[1]

omega_01_A/(h*10**9) , omega_12_A/(h*10**9) 

(1.1516938223021156, 3.2804798038301763)

In [178]:
# mat elements :

n_10_A = np.abs(flxA.mat_elements('charge_op', levels = 5)).data[1,0]
n_01_A = np.abs(flxA.mat_elements('charge_op', levels = 5)).data[0,1]

np.round(np.abs(flxA.mat_elements('charge_op', levels = 5)).data, 2), n_01_A, n_10_A

(array([[0.  , 0.25, 0.  , 0.26, 0.  ],
        [0.25, 0.  , 0.61, 0.  , 0.18],
        [0.  , 0.61, 0.  , 0.75, 0.  ],
        [0.26, 0.  , 0.75, 0.  , 0.94],
        [0.  , 0.18, 0.  , 0.94, 0.  ]]),
 0.24866384335580308,
 0.24866384335580308)

### Rq)

In [161]:
plt.figure()
for i in range(len(bare_E_A)):
    plt.scatter(i, bare_E_A[i])

<IPython.core.display.Javascript object>

In [204]:
flxA.charge_op(as_qobj = True)

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[0.+0.j         0.-0.46530243j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.46530243j 0.+0.j         0.-0.65803701j ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.j         0.+0.65803701j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 ...
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.-4.60625905j 0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+4.60625905j
  0.+0.j         0.-4.62970072j]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.+4.62970072j 0.+0.j        ]]

In [14]:
flxA.charge_zpf*(create(100) - destroy(100))*1j

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[0.+0.j         0.-0.46530243j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.46530243j 0.+0.j         0.-0.65803701j ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.j         0.+0.65803701j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 ...
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.-4.60625905j 0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+4.60625905j
  0.+0.j         0.-4.62970072j]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.+4.62970072j 0.+0.j        ]]

In [145]:
flxA.charge_op(as_qobj = True)

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[0.+0.j         0.-0.46530243j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.46530243j 0.+0.j         0.-0.65803701j ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.j         0.+0.65803701j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 ...
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.-4.60625905j 0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+4.60625905j
  0.+0.j         0.-4.62970072j]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.+4.62970072j 0.+0.j        ]]

In [149]:
flxA.hamiltonian(as_qobj = True)/(h*10**9)

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[ 3.86530968e+00  0.00000000e+00 -1.74179858e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  4.86613613e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.74179858e+00  0.00000000e+00  7.28913515e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.38331779e+02
   0.00000000e+00  2.14719132e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   3.41502781e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  2.14719132e-01
   0.00000000e+00  3.44753251e+02]]

## flx2

$$\hat{H}^{(0)}_B = 4 E_{C,B} \cdot \hat{n}^2_B + \frac{1}{2}E_{L,B}\cdot\hat{\varphi}^2_B - E_{J,B}\cos\left(\hat{\varphi}_B - \phi_{ext,B}\right)$$

In [179]:
e_c_B = 1*10**9 *h
e_l_B = .9*10**9 *h
e_j_B = 3*10**9 *h

flux_B = 1/2

flxB = pysqkit.qubits.Fluxonium(label = "QBB",
                               charge_energy=e_c_B,
                               induct_energy=e_l_B,
                               joseph_energy=e_j_B,
                               flux=flux_B,
                               basis=None,
                               dim_hilbert=N)

In [180]:
eig_enegies, eig_states = flxB.eig_states(levels=5)
flxB.basis.transform(eig_states)

In [182]:
#Bare energies

bare_E_B = flxB.eig_energies()
bare_E_B/(h*10**9)

array([ 2.07847477,  2.92730143,  5.85594017,  8.53943723, 11.62680359,
       14.77621367, 17.91620664, 20.9451284 , 23.78622957, 26.398564  ])

In [183]:
# transitions freqs

omega_01_B = bare_E_B[1] - bare_E_B[0]
omega_12_B = bare_E_B[2] - bare_E_B[1]

omega_01_B/(h*10**9) , omega_12_B/(h*10**9) 

(0.8488266605312955, 2.928638737997042)

In [184]:
# mat elements :

n_10_B = np.abs(flxB.mat_elements('charge_op', levels = 5)).data[1,0]
n_01_B = np.abs(flxB.mat_elements('charge_op', levels = 5)).data[0,1]

np.round(np.abs(flxB.mat_elements('charge_op', levels = 5)).data, 2), n_01_B, n_10_B

(array([[0.  , 0.21, 0.  , 0.28, 0.  ],
        [0.21, 0.  , 0.57, 0.  , 0.17],
        [0.  , 0.57, 0.  , 0.67, 0.  ],
        [0.28, 0.  , 0.67, 0.  , 0.86],
        [0.  , 0.17, 0.  , 0.86, 0.  ]]),
 0.2072248733629435,
 0.20722487336294343)

## coupling

$$\hat{V} = J_C \cdot \hat{n}_A \cdot \hat{n}_B$$

In [185]:
g = J_c#J_c/(4*e**2)

In [186]:
duo = flxA.couple_to(flxB, pysqkit.couplers.capacitive_coupling, strength = g)

In [191]:
duo.eig_energies()/(h*10**9)

array([ 5.20384119,  6.0514322 ,  6.35584493,  7.20120898,  8.98118327,
        9.63508529, 10.10111732, 10.5095348 , 11.66444175, 12.7870574 ])

In [201]:
duo.hamiltonian()

array([[ 3.44826033e-24+0.j, -1.55042123e-62+0.j,  3.35585140e-77+0.j,
        ..., -1.28942341e-28+0.j,  1.30628284e-43+0.j,
         2.55348789e-30+0.j],
       [ 1.55042123e-62+0.j,  4.01069882e-24+0.j, -4.23849168e-62+0.j,
        ...,  2.68670017e-44+0.j, -8.04593391e-29+0.j,
        -2.84611014e-45+0.j],
       [-3.35585140e-77+0.j,  4.23849168e-62+0.j,  5.95123535e-24+0.j,
        ..., -3.12642645e-28+0.j, -1.27012410e-43+0.j,
        -8.15298322e-29+0.j],
       ...,
       [-1.28942341e-28+0.j,  2.68670017e-44+0.j, -3.12642645e-28+0.j,
        ...,  1.78221914e-23+0.j, -4.45705817e-58+0.j,
        -2.16117372e-73+0.j],
       [ 1.30628284e-43+0.j, -8.04593391e-29+0.j, -1.27012410e-43+0.j,
        ...,  4.45705817e-58+0.j,  1.98679019e-23+0.j,
        -5.06926568e-58+0.j],
       [ 2.55348789e-30+0.j, -2.84611014e-45+0.j, -8.15298322e-29+0.j,
        ...,  2.16117372e-73+0.j,  5.06926568e-58+0.j,
         2.19547231e-23+0.j]])

### rqs

In [187]:
duo.bare_hamiltonian(), duo.int_hamiltonian()

(array([[3.44826033e-24+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, ...,
         0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
        [0.00000000e+00+0.j, 4.01069882e-24+0.j, 0.00000000e+00+0.j, ...,
         0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
        [0.00000000e+00+0.j, 0.00000000e+00+0.j, 5.95123535e-24+0.j, ...,
         0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
        ...,
        [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, ...,
         1.78221914e-23+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
        [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, ...,
         0.00000000e+00+0.j, 1.98679019e-23+0.j, 0.00000000e+00+0.j],
        [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, ...,
         0.00000000e+00+0.j, 0.00000000e+00+0.j, 2.19547231e-23+0.j]]),
 array([[ 3.82037620e-96+0.j, -1.55042123e-62+0.j,  3.35585140e-77+0.j,
         ..., -1.28942341e-28+0.j,  1.30628284e-4

In [188]:
#Bare energies

bare_E_duo = la.eig(duo.bare_hamiltonian())[0].real/h #in Hz
bare_E_duo.sort()
bare_E_duo = bare_E_duo[:5]*10**(-9)  #in GHz

int_E_duo = la.eig(duo.int_hamiltonian())[0].real/h
int_E_duo.sort()
int_E_duo = int_E_duo[:5]*10**(-9)

In [189]:
fig, axs = plt.subplots(1,2, figsize = (10,5))

axs[0].scatter(range(len(bare_E_duo)), bare_E_duo, c = 'r', label = 'bare')    
axs[1].scatter(range(len(int_E_duo)), int_E_duo, c = 'b', label = 'int')

axs[0].legend()
axs[1].legend()

axs[0].set_ylabel("E (GHz)")
axs[1].set_ylabel("E (GHz)")

<IPython.core.display.Javascript object>

Text(0, 0.5, 'E (GHz)')

In [31]:
E_duo = bare_E_duo + int_E_duo
E_duo.sort()

plt.figure()
plt.scatter(range(len(E_duo)), E_duo)

plt.annotate("  |00>", (0,E_duo[0]))
plt.annotate("  |01>", (1,E_duo[1]))
plt.annotate("  |10>", (2,E_duo[2]))
plt.annotate("  |11>", (3,E_duo[3]))
plt.annotate("  |??>", (4,E_duo[4]))




plt.ylabel('E (GHz)')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'E (GHz)')

In [32]:
print("Ground =", E_duo[0], "GHz\n1st state =", E_duo[1], "GHz\n2nd state =", E_duo[2], "GHz" )

print("\n\nomega 00->01 =", E_duo[1]-E_duo[0], "GHz\nomega 01->10 =", E_duo[2] - E_duo[1], "GHz\nomega 10->11 =", E_duo[3]-E_duo[2], "GHz"
     )

Ground = 4.586854352151274 GHz
1st state = 5.435681012682565 GHz
2nd state = 5.975780139235088 GHz


omega 00->01 = 0.8488266605312909 GHz
omega 01->10 = 0.5400991265525228 GHz
omega 10->11 = 0.8488266605312917 GHz


In [140]:
duo.hamiltonian()

array([[ 3.44826033e-24+0.j, -1.55042123e-62+0.j,  3.35585140e-77+0.j,
        ..., -1.28942341e-28+0.j,  1.30628284e-43+0.j,
         2.55348789e-30+0.j],
       [ 1.55042123e-62+0.j,  4.01069882e-24+0.j, -4.23849168e-62+0.j,
        ...,  2.68670017e-44+0.j, -8.04593391e-29+0.j,
        -2.84611014e-45+0.j],
       [-3.35585140e-77+0.j,  4.23849168e-62+0.j,  5.95123535e-24+0.j,
        ..., -3.12642645e-28+0.j, -1.27012410e-43+0.j,
        -8.15298322e-29+0.j],
       ...,
       [-1.28942341e-28+0.j,  2.68670017e-44+0.j, -3.12642645e-28+0.j,
        ...,  1.78221914e-23+0.j, -4.45705817e-58+0.j,
        -2.16117372e-73+0.j],
       [ 1.30628284e-43+0.j, -8.04593391e-29+0.j, -1.27012410e-43+0.j,
        ...,  4.45705817e-58+0.j,  1.98679019e-23+0.j,
        -5.06926568e-58+0.j],
       [ 2.55348789e-30+0.j, -2.84611014e-45+0.j, -8.15298322e-29+0.j,
        ...,  2.16117372e-73+0.j,  5.06926568e-58+0.j,
         2.19547231e-23+0.j]])

## Drive

$$\hat{H}_{drive} = 2  f\left( t \right)  \cos \left( \omega_d t + \gamma_d\right) \cdot \left( \eta_A \cdot \hat{n}_A + \eta_B \cdot \hat{n}_B \right)$$

$$f\left( t \right) = f_0 \cdot \dfrac{f\left( t \right)}{f_0}$$

In [213]:
f_0 = lam/2 * np.abs(omega_01_A - omega_01_B) / (n_10_A)

In [214]:
t_rise = 25*10**(-9)
sigma = t_rise/2

f_by_f0 = np.exp(- (t-t_rise)**2 / (2*sigma**2)) - np.exp(-t_rise**2 /(2*sigma**2))

In [215]:
f = f_0 * f_by_f0

omega_d = 2*pi * 1 *10**9
gamma_d = 0

eta_1 = 1
eta_2 = 1

In [216]:
n_A = flxA.charge_op(as_qobj = True)
n_B = flxB.charge_op(as_qobj = True)

In [ ]:
H_drive = []

for i in range(len(t)):
    H_drive.append(2 * f[i] * np.cos(omega_d*t[i]) * (n_1 + n_2))

In [217]:
drive = n_A + n_B

In [218]:
pulse = 2 * f * np.cos(omega_d*t)

### Rq)

In [196]:
n_1+n_2 #Rq) only different parity term

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[0.+0.j         0.-0.87482056j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.87482056j 0.+0.j         0.-1.2371831j  ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.j         0.+1.2371831j  0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 ...
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.-8.66028169j 0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+8.66028169j
  0.+0.j         0.-8.70435466j]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.+8.70435466j 0.+0.j        ]]

In [197]:
n_1*n_2 #Rq) only same parity terms

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[  0.19054978   0.          -0.26947808 ...   0.           0.
    0.        ]
 [  0.           0.57164934   0.         ...   0.           0.
    0.        ]
 [ -0.26947808   0.           0.9527489  ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...  37.15720721   0.
  -18.76891157]
 [  0.           0.           0.         ...   0.          37.53830677
    0.        ]
 [  0.           0.           0.         ... -18.76891157   0.
   18.86442828]]

# Simu

In [220]:
#duo hamil to qobj
dim = N

duo_hamil = Qobj(
                inpt=duo.hamiltonian(),
                dims=[dim, dim],
                shape=duo.hamiltonian(),
                type='oper',
                isherm=True)

In [228]:
time =  t
state_init = fock(N,0)
hamil = duo_hamil
drive = [drive]*len(pulse)
pulse = pulse
jump = []
solver = pysqkit.solvers.solvkit.supported_solvers[0]

In [229]:
res = integrate(time, state_init, hamil, drive, pulse, jump, solver)

TypeError: Incorrect Q_object specification